In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
df = pd.read_csv('V1.csv')

how many samples

In [41]:
df.shape[0]

4000

In [42]:
df.shape

(4000, 14)

avg house price

In [43]:
df.PRICE.mean()

np.float64(24.355923220694248)

How many houses have 5 or more rooms?

In [44]:
(df.RM>=5).sum()


np.int64(3953)

In [45]:
# df.PRICE.nlargest(10).mean()

#or
df.PRICE.sort_values(ascending=False).iloc[:10].mean()

np.float64(52.36590175716407)

What is the total number of missing or unknown values in the number of rooms feature?

In [46]:
df.RM.unique()
(df.RM == -1).sum()

np.int64(40)

In [47]:
df.AGE.unique()
(df.AGE<0).sum()

np.int64(50)

In [48]:
df.RIVERSIDE.unique()
(df.RIVERSIDE == 'UNKNOWN').sum()

np.int64(88)

In [49]:
df[(df.RIVERSIDE == 'YES')&(df.AGE<=50)&(df.AGE>=0)].shape

(44, 14)

In [50]:
df.HIGHWAYCOUNT.isin([6,7,8]).sum()

np.int64(1211)

 Create a column 'CATEGORY' and divide the houses in categories as following:
Category 1: house price <10 lacs
Category 2: 10 lacs <= house price <20 lacs
Category 3: 20 lacs <= house price <30 lacs
Category 4: 30 lacs <= house price <40 lacs
Category 5: house price >=40 lacs
Which category has the highest number of records?

In [51]:
# Solution
def make_cat(price):

  if price<10:
    return "Category 1"

  elif price >= 10 and price < 20:
    return "Category 2"

  elif price >= 20 and price < 30:
    return "Category 3"

  elif price>=30 and price < 40:
    return "Category 4"

  else:
    return "Category 5"


df['Category'] = df.PRICE.apply(make_cat)

In [52]:
df.Category.value_counts().max()

np.int64(2028)

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
df['AGE'].replace({-2:np.nan}, inplace=True)
df['RM'].replace({-1:np.nan}, inplace=True)
df['RIVERSIDE'].replace({'UNKNOWN':np.nan}, inplace=True)
X = df.drop('PRICE', axis=1)
y = df['PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_21364\982319492.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AGE'].replace({-2:np.nan}, inplace=True)
C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_21364\982319492.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [55]:
X_train.shape[0]

2800

Apply following preprocessing steps:
Drop CATEGORY column
CRIM: min max scaling
ZN: min max scaling
INDUS: standard scaling
POLINDEX: min max scaling
DIS: min max scaling
HIGHWAYCOUNT: min max scaling
TAX: min max scaling
PTRATIO: min max scaling
IMM: min max scaling
BPL: min max scaling
RM: impute with median then min max scaling
AGE: impute with mean then min max scaling
RIVERSIDE: Impute with most frequent value then one hot encode.

In [58]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

In [59]:
X_train, X_test = X_train.drop('Category', axis=1), X_test.drop('Category', axis=1)


ct = ColumnTransformer([
    ('mean', SimpleImputer(missing_values=-2), ['AGE']),
    ('median', SimpleImputer(missing_values=-1, strategy='median'), ['RM']),
    ('most_freq', SimpleImputer(missing_values='UNKNOWN', strategy='most_frequent'), ['RIVERSIDE'])
], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')


X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

ct = ColumnTransformer([
    ('minmax', MinMaxScaler(), ['CRIM', 'ZN', 'POLINDEX', 'DIS', 'HIGHWAYCOUNT', 'TAX', 'PTRATIO', 'IMM', 'BPL', 'RM', 'AGE']),
    ('std', StandardScaler(), ['INDUS']),
    ('RIVERSIDE', OneHotEncoder(sparse_output=False), ['RIVERSIDE'])
], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

KeyError: "['Category'] not found in axis"

In [60]:
df1 = pd.read_csv('NPPE1_ModelBuilding3.csv')

In [61]:
X = df1.drop('14', axis=1)
y = df1['14']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [63]:
from sklearn.linear_model import Ridge, Lasso
ridge = Ridge(alpha = 10 ,solver = 'saga', tol = 1e-4, random_state = 42)
ridge.fit(X_train, y_train)
ridge.score(X_test, y_test)

0.6613547575262211

What is the index of most important feature? Note the index starts from 0. Ignore the intercept for this question.

In [64]:
abs(ridge.coef_).argmax()

np.int64(9)

In [65]:
abs(ridge.coef_).argmin()

np.int64(0)

In [68]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

In [69]:
sgd = SGDRegressor(random_state=42)
param_grid = {
    'penalty': ['l1', 'l2'],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1],
}

gscv = GridSearchCV(sgd, param_grid, cv=5, scoring='neg_mean_absolute_error')

gscv.fit(X_train, y_train)


,estimator,SGDRegressor(random_state=42)
,param_grid,"{'alpha': [1e-05, 0.0001, ...], 'penalty': ['l1', 'l2'], 'tol': [0.0001, 0.001, ...]}"
,scoring,'neg_mean_absolute_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'squared_error'


In [70]:
gscv.best_params_

{'alpha': 0.001, 'penalty': 'l2', 'tol': 0.0001}

In [71]:
gscv.score(X_test, y_test)

-3.8131121797994014

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [76]:
pipe = Pipeline([
    ('pca', PCA()),
    ('lasso', Lasso())
])

param_grid = {
    'pca__n_components': [0.9, 0.95],
    'lasso__alpha': [10,1,0.01,0.001]
}

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

,estimator,"Pipeline(step...o', Lasso())])"
,param_grid,"{'lasso__alpha': [10, 1, ...], 'pca__n_components': [0.9, 0.95]}"
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_components,0.95


In [78]:
grid.best_estimator_.score(X_test, y_test)

0.6288625430197583

In [79]:
grid.best_estimator_[0].explained_variance_[0]

np.float64(1.163507574223904)

Create a pipeline of the PolynomialFeatures as transformer and Lasso as an estimator with the following parameters:
For PolynomialFeatures:
interaction_only = False
degree = 2
For Lasso:
alpha = 1
warm_start = True
random state as 0
Fit the pipeline on the training dataset and find the  R2  score on the test dataset.

In [80]:
from sklearn.preprocessing import PolynomialFeatures

In [81]:
# Solution

pipe = Pipeline([

    ('poly', PolynomialFeatures(degree=2, interaction_only=False)),
    ('lasso', Lasso(alpha=1, warm_start=True, random_state=0))
])

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.157678032410551

In [85]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [86]:
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=X_train.shape[1]-1)
rfe.fit(X_train, y_train)

,estimator,LinearRegression()
,n_features_to_select,13
,step,1
,verbose,0
,importance_getter,'auto'
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


Take DecisionTreeRegressor(criterion='absolute_error', random_state=42) with GridSearchCV. Hyperparameter tuning to be done over the following parameters:

'max_depth' as [10, 15, 20, None],
'min_samples_split' as [5, 10, 15],
'max_features' as ['sqrt', 'log2']
Use cross validation = 5
Set scoring as neg_mean_absolute_error
Set n_jobs = -1

In [87]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(criterion='absolute_error', random_state=42)
param_grid={
    'max_depth': [10,15,20,None],
    'min_samples_split': [5,10,15],
    'max_features': ['sqrt', 'log2']
}

gscv = GridSearchCV(dt, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
gscv.fit(X_train, y_train)

,estimator,DecisionTreeR...ndom_state=42)
,param_grid,"{'max_depth': [10, 15, ...], 'max_features': ['sqrt', 'log2'], 'min_samples_split': [5, 10, ...]}"
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'absolute_error'


What is the best max_depth?

In [88]:
gscv.best_params_

{'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 5}

Using the best estimator, find the  R2  score on the test dataset

In [89]:
gscv.best_estimator_.score(X_test, y_test)

0.7768079393414741

What is the height of the best tree?

In [90]:
gscv.best_estimator_.get_depth()

32

How many nodes are there in the best tree?

In [91]:
gscv.best_estimator_.tree_.node_count

2541

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(10,12))
plot_tree(gscv.best_estimator_, max_depth=3, rounded=True, fontsize=12, filled=True)
plt.show()